# Designing and Creating a Database

### First investigate the data fields

In [1]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


### Setup environment and import data

In [2]:
import pandas as pd
import sqlite3

#set display parameters due to large data size
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [3]:
game_log = pd.read_csv('game_log.csv',low_memory=False)
print(game_log.shape)

(171907, 161)


In [4]:
print(game_log.head())

       date  number_of_game day_of_week v_name v_league  v_game_number h_name  \
0  18710504               0         Thu    CL1      NaN              1    FW1   
1  18710505               0         Fri    BS1      NaN              1    WS3   
2  18710506               0         Sat    CL1      NaN              2    RC1   
3  18710508               0         Mon    CL1      NaN              3    CH1   
4  18710509               0         Tue    BS1      NaN              2    TRO   

  h_league  h_game_number  v_score  h_score  length_outs day_night completion  \
0      NaN              1        0        2         54.0         D        NaN   
1      NaN              1       20       18         54.0         D        NaN   
2      NaN              1       12        4         54.0         D        NaN   
3      NaN              1       12       14         54.0         D        NaN   
4      NaN              1        9        5         54.0         D        NaN   

  forefeit protest park_id

In [5]:
print(game_log.tail())

            date  number_of_game day_of_week v_name v_league  v_game_number  \
171902  20161002               0         Sun    MIL       NL            162   
171903  20161002               0         Sun    NYN       NL            162   
171904  20161002               0         Sun    LAN       NL            162   
171905  20161002               0         Sun    PIT       NL            162   
171906  20161002               0         Sun    MIA       NL            161   

       h_name h_league  h_game_number  v_score  h_score  length_outs  \
171902    COL       NL            162        6        4         60.0   
171903    PHI       NL            162        2        5         51.0   
171904    SFN       NL            162        1        7         51.0   
171905    SLN       NL            162        4       10         51.0   
171906    WAS       NL            162        7       10         51.0   

       day_night completion forefeit protest park_id  attendance  \
171902         D        

### Description of `game_log`
The data in `game_log` represents each of the game played between 1871/05/04 and 2016/10/02 - for a total of 171906 games.

The columns reflects the `game_log_fields.txt` that we explored earlier.

The data also recorded the defensive positions for the starting players with the following:
1 (pitcher), 2 (catcher), 3 (first baseman), 4 (second baseman), 5 (third baseman), 6 (shortstop), 7 (left fielder), 8 (center fielder), and 9 (right fielder)
___

In [6]:
park_codes = pd.read_csv('park_codes.csv')

In [7]:
print(park_codes.shape)

(252, 9)


In [8]:
print(park_codes.head())

  park_id                           name  \
0   ALB01                 Riverside Park   
1   ALT01                  Columbia Park   
2   ANA01       Angel Stadium of Anaheim   
3   ARL01              Arlington Stadium   
4   ARL02  Rangers Ballpark in Arlington   

                                        aka       city state       start  \
0                                       NaN     Albany    NY  09/11/1880   
1                                       NaN    Altoona    PA  04/30/1884   
2             Edison Field; Anaheim Stadium    Anaheim    CA  04/19/1966   
3                                       NaN  Arlington    TX  04/21/1972   
4  The Ballpark in Arlington; Ameriquest Fl  Arlington    TX  04/11/1994   

          end league                                           notes  
0  05/30/1882     NL  TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882  
1  05/31/1884     UA                                             NaN  
2         NaN     AL                                             

In [9]:
print(park_codes.tail())

    park_id                                 name  aka           city state  \
247   WIL02        BB&T Ballpark at Bowman Field  NaN    Wiliamsport    PA   
248   WNY01     West New York Field Club Grounds  NaN  West New York    NJ   
249   WOR01   Agricultural County Fair Grounds I  NaN      Worcester    MA   
250   WOR02  Agricultural County Fair Grounds II  NaN      Worcester    MA   
251   WOR03       Worcester Driving Park Grounds  NaN      Worcester    MA   

          start         end league  \
247  08/20/2017  08/20/2017     NL   
248  09/11/1898  09/17/1899     NL   
249  05/01/1880  09/29/1882     NL   
250  08/17/1887  08/17/1887     NL   
251  10/30/1874  10/30/1874    NaN   

                                                      notes  
247                                                     PIT  
248  BRO:9/18&10/2/1898; NY1:9/11/98, 6/4&7/16&8/13&9/17/99  
249                                                     NaN  
250                                              1 BSN

### Description of `park_codes`
The data in `park_codes` represents the information for each of the baseball park - names, cities, states, dates in operation, and which leagues they belong to.

There are 251 parks within the data.

The data can be linked with the game_log dataframe via the `park_id` column.
___

In [10]:
person_codes = pd.read_csv('person_codes.csv')

In [11]:
print(person_codes.shape)

(20494, 7)


In [12]:
print(person_codes.head())

         id     last   first player_debut mgr_debut coach_debut ump_debut
0  aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
1  aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
2  aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
3  aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
4  abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN


In [13]:
print(person_codes.tail())

             id      last   first player_debut mgr_debut coach_debut ump_debut
20489  zuvep001   Zuvella    Paul   09/04/1982       NaN  04/02/1996       NaN
20490  zuveg101  Zuverink  George   04/21/1951       NaN         NaN       NaN
20491  zwild101  Zwilling   Dutch   08/14/1910       NaN  04/15/1941       NaN
20492  zycht001      Zych    Tony   09/04/2015       NaN         NaN       NaN
20493  thoma102  Thompson     NaN          NaN       NaN         NaN       NaN


### Description of `person_codes`
The data holds the id, name, debut date (as player, manager, coach, or umpire), for a total of 20493 people.

The data can be linked to the `game_log` via the `id` column in `person_code` and the various players/coaches/umpires fields in `game_log`.
___

In [14]:
team_codes = pd.read_csv('team_codes.csv')

In [15]:
print(team_codes.shape)

(150, 8)


In [16]:
print(team_codes.head())

  team_id league  start   end       city         nickname franch_id  seq
0     ALT     UA   1884  1884    Altoona  Mountain Cities       ALT    1
1     ARI     NL   1998     0    Arizona     Diamondbacks       ARI    1
2     BFN     NL   1879  1885    Buffalo           Bisons       BFN    1
3     BFP     PL   1890  1890    Buffalo           Bisons       BFP    1
4     BL1    NaN   1872  1874  Baltimore         Canaries       BL1    1


In [17]:
print(team_codes.tail())

    team_id league  start   end        city   nickname franch_id  seq
145     WS8     NL   1886  1889  Washington   Senators       WS8    1
146     WS9     AA   1891  1891  Washington   Senators       WS9    1
147     WSN     NL   1892  1899  Washington   Senators       WS9    2
148     WSU     UA   1884  1884  Washington  Nationals       WSU    1
149     MIA     NL   2012     0       Miami    Marlins       FLO    2


### Description of `team_codes`
The data in `team_codes` holds the name, league, active years, city, and id, for a total of 149 team.

The `team_id` column in `team_codes` can be linked with various team related fields (visiting team, home team) in `game_log`
___

### Setup SQLite3 environment and functions

In [18]:
# define database filename
db = 'mlb.db'

# a function to run query directly
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

# a function to run and not return the query
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

### Create tables in the database

In [19]:
with sqlite3.connect(db) as conn:
    conn.execute("DROP TABLE IF EXISTS {};".format('game_log'))
    game_log.to_sql('game_log', conn, index=False)

In [20]:
with sqlite3.connect(db) as conn:
    conn.execute("DROP TABLE IF EXISTS {};".format('park_codes'))
    conn.execute("DROP TABLE IF EXISTS {};".format('person_codes'))
    conn.execute("DROP TABLE IF EXISTS {};".format('team_codes'))
    park_codes.to_sql('park_codes', conn, index=False)
    person_codes.to_sql('person_codes', conn, index=False)
    team_codes.to_sql('team_codes', conn, index=False)

In [21]:
# check tables are created
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table


### Create unique `game_id` for the table `game_log`

In [22]:
# create the column (no values yet)
run_command('''
ALTER TABLE game_log
ADD COLUMN game_id TEXT;
''')

In [23]:
# populate with values
q = ('''
UPDATE game_log
SET
    game_id = date || h_name || number_of_game
''')
run_command(q)

In [24]:
# check the game_id is properly created
q1 = ('''
SELECT
    game_id,
    h_name,
    date,
    number_of_game
FROM game_log
LIMIT 10;
''')

run_query(q1)

,game_id,h_name,date,number_of_game
0,18710504FW10,FW1,18710504,0
1,18710505WS30,WS3,18710505,0
2,18710506RC10,RC1,18710506,0
3,18710508CH10,CH1,18710508,0
4,18710509TRO0,TRO,18710509,0
5,18710511CL10,CL1,18710511,0
6,18710513CL10,CL1,18710513,0
7,18710513FW10,FW1,18710513,0
8,18710515FW10,FW1,18710515,0
9,18710516BS10,BS1,18710516,0


___
### Database Normalization
We want to build a database from the four dataframes - 
- `game_log`
- `park_codes`
- `person_codes`
- `team_codes`

Among these tables, we noticed that there are opportunities for normalization:
#### `game_log`
- `v_league` / `h_league` is redundant, as already present in `team_codes`
- fields relating to the names of umpires, managers, and players are redundant, as already present in `person_codes`
- players' names and defensive positions are recorded over multiple iterations
- the above observation also applies to umpire related fields
- the data granualarity is set to team level and lost at the player level

#### `park_codes`
- `start` and `end` can be derived from `game_log`
- `league` can be derived from `game_log` and `team_codes`

#### `person_codes`
- the various debut fields can be derived from `game_log`

#### `team_codes`
- `start` and `end` can be derived from `game_log`

### Proposed Schema
With the above observations in mind, this is the proposed schema for the database:
![Proposed schema](http://clarkhsu.info/wp-content/uploads/2018/03/mlb_schema_revised.png)
___

#### Create and populate `person` table

In [25]:
# create `person` table
construct_person = '''
CREATE TABLE IF NOT EXISTS person (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT);
'''

# populate `person` with data from `person_codes`
populate_person = '''
INSERT OR IGNORE INTO person
SELECT
    id,
    first,
    last
FROM person_codes;
'''

# execute
try:
    run_command(construct_person)
except:
    print('construction failed')

try:
    run_command(populate_person)
except:
    print('populate person failed')

In [26]:
# check `person` table
check_person = ('''
SELECT *
FROM person
LIMIT 10
''')
run_query(check_person)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad
5,abadf001,Fernando,Abad
6,abadj101,John,Abadie
7,abbae101,Ed,Abbaticchio
8,abbeb101,Bert,Abbey
9,abbec101,Charlie,Abbey


#### Create and populate `park` table

In [27]:
# create `park` table
construct_park = '''
CREATE TABLE IF NOT EXISTS park (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT);
'''

# populate `park` with data from `park_codes`
populate_park = '''
INSERT OR IGNORE INTO park
SELECT
    park_id,
    name,
    aka,
    city,
    state,
    notes
FROM park_codes;
'''

# execute
try:
    run_command(construct_park)
except:
    print('construction failed')

try:
    run_command(populate_park)
except:
    print('populate park failed')

In [28]:
# check `park` table
check_park = ('''
SELECT *
FROM park
LIMIT 10
''')
run_query(check_park)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None
5,ATL01,Atlanta-Fulton County Stadium,None,Atlanta,GA,None
6,ATL02,Turner Field,None,Atlanta,GA,None
7,ATL03,Suntrust Park,None,Atlanta,GA,None
8,BAL01,Madison Avenue Grounds,None,Baltimore,MD,WS3
9,BAL02,Newington Park,None,Baltimore,MD,BL1:1872-74; BL4:1873; BL2: 1882


#### Create and populate `league` table

In [29]:
park_codes['league'].value_counts()

NL    88
AL    47
AA    33
UA     9
FL     7
PL     2
Name: league, dtype: int64

In [30]:
# create `league` table
construct_league = '''
CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT);
'''

# populate `league` with manual data insertion
populate_league = '''
INSERT OR IGNORE INTO league
VALUES
    ('NL','National League'),
    ('AL','American League'),
    ('AA','American Association'),
    ('UA','Union Association'),
    ('FL','Federal League'),
    ('PL','Players League');
'''

# execute
try:
    run_command(construct_league)
except:
    print('construction failed')

try:
    run_command(populate_league)
except:
    print('populate league failed')

In [31]:
# check `league` table
check_league = ('''
SELECT *
FROM league
''')
run_query(check_league)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,American Association
3,UA,Union Association
4,FL,Federal League
5,PL,Players League


#### Create and populate `appearance_type` table

In [32]:
appearance_type = pd.read_csv('appearance_type.csv')
print(appearance_type.shape)
print(appearance_type.head())

(31, 3)
  appearance_type_id      name category
0                 O1  Batter 1  offense
1                 O2  Batter 2  offense
2                 O3  Batter 3  offense
3                 O4  Batter 4  offense
4                 O5  Batter 5  offense


In [33]:
# create `appearance_type` table
# populate with data from appearance_type dataframe

run_command('DROP TABLE IF EXISTS appearance_type;')

construct_appearance_type = '''
CREATE TABLE appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT);
'''
run_command(construct_appearance_type)

appearance_type = pd.read_csv('appearance_type.csv')

with sqlite3.connect(db) as conn:
    appearance_type.to_sql('appearance_type',
                           conn,
                           index=False,
                           if_exists='append')

In [34]:
# check `appearance_type` table
check_apperance_type = ('''
SELECT *
FROM appearance_type
LIMIT 10
''')
run_query(check_apperance_type)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


#### Create and populate `team` table

In [35]:
# create `team` table
construct_team = '''
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id));
'''

# populate `team` with data from `team_codes`
populate_team = '''
INSERT OR IGNORE INTO team
SELECT
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes;
'''

# execute
try:
    run_command(construct_team)
except:
    print('construction failed')

try:
    run_command(populate_team)
except:
    print('populate team failed')

In [36]:
# check `team` table
check_team = ('''
SELECT *
FROM team
LIMIT 10
''')
run_query(check_team)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1
5,BL2,AA,Baltimore,Orioles,BL2
6,BLN,NL,Baltimore,Orioles,BL2
7,BL4,None,Baltimore,Marylands,BL4
8,BLA,AL,Baltimore,Orioles,BLA
9,NYA,AL,New York,Yankees,BLA


In [37]:
# create `game` table
construct_game = '''
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id));
'''

# populate `game` with data from `game_log`
populate_game = '''
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE
        WHEN day_night = "D" THEN 1
        WHEN day_night = "N" THEN 0
        ELSE NULL
        END
        AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
'''

# execute
try:
    run_command(construct_game)
except:
    print('construction failed')

try:
    run_command(populate_game)
except:
    print('populate game failed')

In [38]:
# check `game` table
check_game = ('''
SELECT *
FROM game
LIMIT 10
''')
run_query(check_game)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,18710504FW10,18710504,0,FOR01,54,1,None,None,None,200.0,120.0,None,Y
1,18710505WS30,18710505,0,WAS01,54,1,None,None,None,5000.0,145.0,HTBF,Y
2,18710506RC10,18710506,0,RCK01,54,1,None,None,None,1000.0,140.0,None,Y
3,18710508CH10,18710508,0,CHI01,54,1,None,None,None,5000.0,150.0,None,Y
4,18710509TRO0,18710509,0,TRO01,54,1,None,None,None,3250.0,145.0,HTBF,Y
5,18710511CL10,18710511,0,CLE01,48,1,None,V,None,2500.0,120.0,None,Y
6,18710513CL10,18710513,0,CIN01,54,1,None,None,None,1200.0,150.0,None,Y
7,18710513FW10,18710513,0,FOR01,54,1,None,None,None,1500.0,105.0,None,Y
8,18710515FW10,18710515,0,FOR01,54,1,None,None,None,NaN,140.0,None,Y
9,18710516BS10,18710516,0,BOS01,54,1,None,None,None,2500.0,NaN,HTBF,Y


In [39]:
# create `team_appearance` table
construct_team_appearance = '''
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id));
'''

# populate `team_apperaance` with data from `game_log`
populate_team_appearance = '''
INSERT OR IGNORE INTO team_appearance
SELECT
    h_name,
    game_id,
    1 AS home,
    h_league,
    h_score,
    h_line_score,
    h_at_bats,
    h_hits,
    h_doubles,
    h_triples,
    h_homeruns,
    h_rbi,
    h_sacrifice_hits,
    h_sacrifice_flies,
    h_hit_by_pitch,
    h_walks,
    h_intentional_walks,
    h_strikeouts,
    h_stolen_bases,
    h_caught_stealing,
    h_grounded_into_double,
    h_first_catcher_interference,
    h_left_on_base,
    h_pitchers_used,
    h_individual_earned_runs,
    h_team_earned_runs,
    h_wild_pitches,
    h_balks,
    h_putouts,
    h_assists,
    h_errors,
    h_passed_balls,
    h_double_plays,
    h_triple_plays
FROM game_log

UNION

SELECT    
    v_name,
    game_id,
    0 AS home,
    v_league,
    v_score,
    v_line_score,
    v_at_bats,
    v_hits,
    v_doubles,
    v_triples,
    v_homeruns,
    v_rbi,
    v_sacrifice_hits,
    v_sacrifice_flies,
    v_hit_by_pitch,
    v_walks,
    v_intentional_walks,
    v_strikeouts,
    v_stolen_bases,
    v_caught_stealing,
    v_grounded_into_double,
    v_first_catcher_interference,
    v_left_on_base,
    v_pitchers_used,
    v_individual_earned_runs,
    v_team_earned_runs,
    v_wild_pitches,
    v_balks,
    v_putouts,
    v_assists,
    v_errors,
    v_passed_balls,
    v_double_plays,
    v_triple_plays
FROM game_log;
'''

# execute
try:
    run_command(construct_team_appearance)
except:
    print('construction failed')

try:
    run_command(populate_team_appearance)
except:
    print('populate team_appearance failed')

In [40]:
# check `team_appearance` table
run_query('''
SELECT *
FROM team_appearance
LIMIT 5''')

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,18840417CNU0,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,18840418CNU0,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT,18840419CNU0,0,UA,6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT,18840424SLU0,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT,18840426SLU0,0,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [42]:
# create `person_appearance` table
run_command('DROP TABLE IF EXISTS person_appearance')

construct_person_appearance = '''
CREATE TABLE IF NOT EXISTS person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
'''

# populate `person_apperaance` with data from `game_log`
populate_person_appearance_non_players = ('''
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
''')

populate_person_appearance_players = ('''
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
''')

# execute
try:
    run_command(construct_person_appearance)
except:
    print('construction failed')

try:
    run_command(populate_person_appearance_non_players)
except:
    print('populate person_appearance_non_players failed')
    
for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        try:
            run_command(populate_person_appearance_players.format(**query_vars))
        except:
            print('populate person_appearance_players failed')

In [47]:
# check the person_appearance table by comparing the number of unique game_id
# between game table vs person_appearance table
print(
    run_query('''
SELECT COUNT(DISTINCT game_id)
FROM game
''')
)

print(
    run_query('''
SELECT COUNT(DISTINCT game_id)
FROM person_appearance
''')
)

# check the person_appearance table by constructing a table that shows all
# the participating umpires/coaches/players
run_query('''
SELECT
    pa.game_id,
    pa.team_id,
    pa.person_id,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at ON at.appearance_type_id = pa.appearance_type_id
WHERE game_id = (SELECT MAX(game_id) FROM person_appearance)
ORDER BY team_id, at.category, at.name
''')


   COUNT(DISTINCT game_id)
0                   171907
   COUNT(DISTINCT game_id)
0                   171907


,game_id,team_id,person_id,name,category
0,20161002WAS0,None,porta901,First Base,umpire
1,20161002WAS0,None,tumpj901,Home Plate,umpire
2,20161002WAS0,None,onorb901,Second Base,umpire
3,20161002WAS0,None,kellj901,Third Base,umpire
4,20161002WAS0,MIA,brica001,Losing Pitcher,award
5,20161002WAS0,MIA,bourj002,1st Base,defense
6,20161002WAS0,MIA,gordd002,2nd Base,defense
7,20161002WAS0,MIA,pradm001,3rd Base,defense
8,20161002WAS0,MIA,telit001,Catcher,defense
9,20161002WAS0,MIA,yelic001,Center Field,defense


In [52]:
# drop tables that are not being used for the final database
# ie: the original tables used for populating the newly normalized database
drop_table = ["game_log", "park_codes", 'team_codes', 'person_codes']
for t in drop_table:
        command = 'DROP TABLE {}'.format(t)
        run_command(command)

In [53]:
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table
